In [4]:
%reload_ext autoreload
%autoreload 2

import torch
from tqdm.auto import tqdm


import eval
import mmlu

In [5]:
checkpoints = ["google/flan-t5-large"]  # for testing
# checkpoints = [
#     "google/flan-t5-small",
#     "google/flan-t5-base",
#     "google/flan-t5-large",
#     "google/flan-t5-xl",
#     "google/flan-t5-xxl",
# ]


In [6]:
for checkpoint in checkpoints:
    for cot in [False, True]:
        dataset_results = []
        for task in mmlu.task_list:
            config = eval.EvalConfig(
                checkpoint=checkpoint,
                dataset_name="lukaemon/mmlu",
                task_name=task,
                split="test",
                cot=cot,
                n=5,  # mmlu uses 5 shots
                cutoff=8,
                batch_size=2,
                temp=0.0,
                k=1,
                inst_fn=mmlu.task_instruction,
                cot_fn=mmlu.task_cot,
            )

            task_result = eval.eval_task(config)
            dataset_results.append(task_result)

        eval.save_result(dataset_results)
    torch.cuda.empty_cache()


[INFO] [2023-02-02 04:38:05,239] [flan-t5] [eval.py:69] Loading model from google/flan-t5-large.
[INFO] [2023-02-02 04:38:15,315] [flan-t5] [eval.py:141] Saving accuracy result to output/mmlu/accuracy/flan-t5-large_cot_False_k_1.csv.
[INFO] [2023-02-02 04:38:15,317] [flan-t5] [eval.py:151] Saving failed output to output/mmlu/failed/flan-t5-large.


Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

[INFO] [2023-02-02 04:38:15,332] [flan-t5] [eval.py:69] Loading model from google/flan-t5-large.
[INFO] [2023-02-02 04:38:32,104] [flan-t5] [eval.py:141] Saving accuracy result to output/mmlu/accuracy/flan-t5-large_cot_True_k_1.csv.
[INFO] [2023-02-02 04:38:32,106] [flan-t5] [eval.py:151] Saving failed output to output/mmlu/failed/flan-t5-large.


Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]